# Permutation Feature Imporance

From this guide: https://scikit-learn.org/stable/modules/permutation_importance.html

## Overview

Permutation feature importance is a model inspection technique that can be used for **any fitted estimator** when the *data is tabular*. This is especially useful for *non-linear or opaque estimators*.

The permutation feature importance is defined to be:
> the decrease in a model score when a single feature value is randomly shuffled.

* This procedure **breaks the relationship between the feature and the target**, thus the drop in the model score is indicative of **how much the model depends on the feature**.
* This technique benefits from being model agnostic and can be calculated many times with different permutations of the feature.

The `permutation_importance` *function* calculates the feature importance of *estimators* for a given *dataset*. The `n_repeats` parameter sets the number of times a feature is randomly shuffled and returns *a sample of feature importances*.

Let’s consider the following trained regression model:

In [1]:
from IPython.display import display
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge

In [12]:
diabetes = load_diabetes()
X_train, X_val, y_train, y_val = train_test_split(
    diabetes.data, 
    diabetes.target, 
    random_state=0
)

In [13]:
X_train.shape

(331, 10)

In [3]:
model = Ridge(alpha=1e-2).fit(X_train, y_train)
model.score(X_val, y_val)

0.3566606238695452

Its validation performance, measured via the $R^2$ **score**, is significantly larger than the chance level. This makes it possible to use the `permutation_importance` function to probe which features are most predictive:

In [4]:
from sklearn.inspection import permutation_importance

In [7]:
r = permutation_importance(
    model, 
    X_val, 
    y_val,
    n_repeats=30,
    random_state=0
)

print(type(r))

<class 'sklearn.utils.Bunch'>


In [9]:
# 💡 Side note: see here what the object returned by `permutation_importance()` is...
# It is NOT an estimator, it is basically a dictionary.
import inspect
inspect.getmro(type(r))

(sklearn.utils.Bunch, dict, object)

In [15]:
print(r.importances_mean.shape)  # Note: the shape is 10, for 10 features!
r.importances_mean  # A (mean) importance value for each feature.

(10,)


array([-0.00199267,  0.05587407,  0.17579   ,  0.08836513,  0.04221134,
        0.00203626,  0.00203754,  0.00318695,  0.20423412,  0.00278683])

In [16]:
for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        # ^ NOTE: 
        # Choose only those featurese where mean_importance - 2 * std_of_imporance > 0...
        print(f"{diabetes.feature_names[i]:<8}"
              f"{r.importances_mean[i]:.3f}"
              f" +/- {r.importances_std[i]:.3f}")

s5      0.204 +/- 0.050
bmi     0.176 +/- 0.048
bp      0.088 +/- 0.033
sex     0.056 +/- 0.023


Note that the importance values for the top features *represent a large fraction of the reference score* of `0.356`.

⚠️ Permutation importances **can be computed either on the training set or on a held-out testing or validation set**. Using a held-out set makes it possible to highlight which features contribute the most to the generalization power of the inspected model. Features that are important on the training set but not on the held-out set might cause the model to overfit.

---

**⚠️ Warning:**

Features that are deemed of **low importance for a *bad model*** (low cross-validation score) could be **very important for a *good model***. 

‼️ Therefore it is **always important to evaluate the `predictive power of the *model*` using a *held-out set* (or better with *cross-validation*)** *prior to computing importances*. 

Permutation importance does not reflect to the intrinsic predictive value of a feature by itself but how important this feature is for a particular model. 

---

## [Outline of the permutation importance algorithm](https://scikit-learn.org/stable/modules/permutation_importance.html#outline-of-the-permutation-importance-algorithm)

📚🎓 See the [section](https://scikit-learn.org/stable/modules/permutation_importance.html#outline-of-the-permutation-importance-algorithm)

## [Relation to impurity-based importance in trees](https://scikit-learn.org/stable/modules/permutation_importance.html#relation-to-impurity-based-importance-in-trees)

Tree-based models provide an **alternative measure of feature importances** based on the [mean decrease in impurity (MDI)](https://scikit-learn.org/stable/modules/ensemble.html#random-forest-feature-importance).

Impurity is quantified by the **splitting criterion of the decision trees** (`Gini`, `Entropy` or `Mean Squared Error`). However, this method can give high importance to features that may not be predictive on unseen data *when the model is overfitting*. Permutation-based feature importance, on the other hand, *avoids this issue, since it can be computed on unseen data*.

Furthermore, impurity-based feature importance for trees are **strongly biased** and **favor high cardinality features** (typically numerical features) over low cardinality features such as binary features or categorical variables with a small number of possible categories. Permutation-based feature importances *do not exhibit such a bias*. 

The following example highlights the limitations of impurity-based feature importance in contrast to permutation-based feature importance: [📚 Permutation Importance vs Random Forest Feature Importance (MDI)](https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance.html#sphx-glr-auto-examples-inspection-plot-permutation-importance-py).

## [Misleading values *on strongly correlated features*](https://scikit-learn.org/stable/modules/permutation_importance.html#misleading-values-on-strongly-correlated-features)

**⚠️‼️ When two features are correlated and one of the features is permuted, the model will still have access to the feature through its correlated feature.** This will result in a **lower importance value for both features**, where **they might actually be important**.

One way to handle this is to *cluster features that are correlated and only keep one feature from each cluster*. This strategy is explored in the following example: [📚 Permutation Importance with Multicollinear or Correlated Features](https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance_multicollinear.html#sphx-glr-auto-examples-inspection-plot-permutation-importance-multicollinear-py).

**📚 Examples:**
* [Permutation Importance vs Random Forest Feature Importance (MDI)](https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance.html#sphx-glr-auto-examples-inspection-plot-permutation-importance-py)
* [Permutation Importance with Multicollinear or Correlated Features](https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance_multicollinear.html#sphx-glr-auto-examples-inspection-plot-permutation-importance-multicollinear-py)